In [1]:
import sys
import os
import yaml

# Add the current directory to sys.path to ensure we can import the package
sys.path.append(os.getcwd())

from uav_mission_env.utils.schema_utils import create_gbnf_grammar
import numpy as np

In [2]:
# Load output_keys and tools from minimal_viable_states.yaml
config_path = 'uav_mission_env/configs/minimal_viable_states.yaml'

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Extract output_keys and tools from the 'execution' state
execution_keys = config['states']['execution']['output_keys']
execution_tools = config['states']['execution']['tools']

# Extract output_keys and tools from the 'conclusion_generation' state
conclusion_keys = config['states']['conclusion_generation']['output_keys']
conclusion_tools = config['states']['conclusion_generation']['tools']

print("Loaded configuration:")
print("Execution Keys:", execution_keys)
print("Execution Tools:", execution_tools)
print("Conclusion Keys:", conclusion_keys)
print("Conclusion Tools:", conclusion_tools)

Loaded configuration:
Execution Keys: [{'justification': {'type': 'string', 'max_length': 300}}, {'tool_call': {'type': 'object', 'max_length': 400}}, {'information': {'type': 'string', 'max_length': 300}}]
Execution Tools: ['next_goal']
Conclusion Keys: [{'justification': {'type': 'string', 'max_length': 300}}, {'tool_call': {'type': 'object'}}]
Conclusion Tools: ['report_final_conclusion']


In [ ]:
# Run the create_gbnf_grammar function with loaded keys and tools

print("--- Execution State Grammar ---")
execution_grammar = create_gbnf_grammar(execution_keys, execution_tools)
print(execution_grammar)

print("\n" + "="*50 + "\n")

print("--- Conclusion Generation State Grammar ---")
conclusion_grammar = create_gbnf_grammar(conclusion_keys, conclusion_tools)
print(conclusion_grammar.replace("__THINK_LIMIT__", "2500"))  # Example replacement for think limit

--- Execution State Grammar ---
?start: thinking? json_output
thinking: "<think>" /[\s\S]{0,__THINK_LIMIT__}?/ "</think>" "\n"?
json_output: "{" field_justification "," field_tool_call "," field_information "}"
field_justification: "\" justification \":" string_300
field_tool_call: "\" tool_call \": tool_obj
field_information: "\" information \":" string_300
param_obj: "{" /[\s\S]*?/ "}"
string_300: "\"" /[^"\n]{0,300}/ "\""
tool_name: "\"next_goal\""
tool_obj: "{" "\"name\":" tool_name "," "\"parameters\":" param_obj "}"


--- Conclusion Generation State Grammar ---
?start: thinking? json_output
thinking: "<think>" /[\s\S]{0,__THINK_LIMIT__}?/ "</think>" "\n"?
json_output: "{" field_justification "," field_tool_call "}"
field_justification: "\" justification \":" string_300
field_tool_call: "\" tool_call \": tool_obj
param_obj: "{" /[\s\S]*?/ "}"
string_300: "\"" /[^"\n]{0,300}/ "\""
tool_name: "\"report_final_conclusion\""
tool_obj: "{" "\"name\":" tool_name "," "\"parameters\":" par